# Notebook: Split Dataset in folds

## Packages

In [121]:
from llama_cpp import Llama
import numpy as np
import itertools
import warnings
import random

## Settings

In [122]:
SPLIT = 0

# Select Model for Synthesis
MODELS = ["Llama70B"]
MODEL = MODELS[0]

# Number of Examples to be generated with the LLM
N_SYNTH = 500

# Setup Classes/Polarities for Synthesis
CLASSES  = ["GENERAL_IMPRESSION", "FOOD", "SERVICE", "AMBIENCE", "PRICE"]
POLARITY = ["POSITIVE", "NEUTRAL", "NEGATIVE"]

## Parameters

In [123]:
DATASET_PATH = f'../03 dataset split/real_{SPLIT}'
MAX_TOKENS = 200
CONTEXT_SIZE = 2048
SEED = 43

In [124]:
random.seed(SEED)

## Code

In [209]:
def divide_equally(x):
    equally = x // 3
    remainder = x % 3
    values = [equally, equally, equally]

    for i in range(remainder):
        values[i] += 1

    # Zufällige Anordnung der Werte
    random.shuffle(values)

    return values

n_for_n_aspects = divide_equally(500)
n_aspects_synth = sum([n_for_n_aspects[0], n_for_n_aspects[1] * 2, n_for_n_aspects[2] * 3])

In [210]:
n_aspects_synth, n_for_n_aspects

(1000, [167, 166, 167])

In [251]:
combinations = list(itertools.product(CLASSES, POLARITY))
total_combinations = len(combinations)
desired_count_per_combination = n_aspects_synth // total_combinations
remaining_tuples = n_aspects_synth % total_combinations
tuples = []
for combination in combinations:
    for _ in range(desired_count_per_combination):
        tuples.append(combination)

if remaining_tuples > 0:
    tuples.extend(combinations[:remaining_tuples])
    
random.shuffle(tuples)

In [252]:
len(tuples)

1000

In [257]:
n_aspects = 1
labels = []
idx_aspects = 0

for k in n_for_n_aspects:
    for n in range(k):
        label = []
        for aspect_idx in range(n_aspects):
            # hier aspekt einfügen
            label.append(tuples[idx_aspects])
            idx_aspects += 1
        labels.append(label)
        #print(k, n, n_aspects, label)
    n_aspects += 1

### Setup Prompt Template

In [43]:
with open('../prompt_template.txt', 'r') as file:
    PROMPT_TEMPLATE = file.read()

In [44]:
print(PROMPT_TEMPLATE)

Bitte erzeuge genau einen Satz einer Restaurantbewertung, die für das Training eines Modells für die Aspekt-basierte Sentiment Analyse verwendet werden kann.
Gegeben ist ein Label in Form eines Arrays, wobei für jedes Sentiment, mit dem eine Aspekt-Kategorie im Satz adressiert wird, ein Tuple (Aspekt-Kategorie, Aspekt-Sentiment) vorhanden ist.
Eine Aspekt-Kategorie kann mehrfach im Satz adressiert werden, auch mit verschiedenen Sentiment-Polaritäten.

* Aspekt-Kategorien hat nur diese Werte: ["GENERAL_IMPRESSION", "FOOD", "SERVICE", "PRICE", "AMBIENCE"]
* Sentiment nur diese Polaritäten: ["negative", "neutral", "positive"]

Auf Basis des Arrays soll genau ein deutscher Satz erzeugt werden, der die in den Tuple definierten Aspekte enthält.
Zusätzlich kann für ein im Label vorgegebenes Tuple (Aspekt-Kategorie, Aspekt-Sentiment) ein Aspekte-Term im Text vorliegen.
Der Aspekt-Term wird mithilfe eines xml-Tags markiert.

Gebe nur die Prediction zurück, ohne Kommentare oder zusätzlichen Text

### Iterate over Dataset

In [46]:
N_SAMPLES_FOR_COMBINATION = N_SYNTH / (len(CLASSES) + len(POLARITY))
N_SAMPLES_FOR_COMBINATION

62.5

### Load Dataset

In [ ]:
with open(DATASET_PATH, 'r', encoding='utf-8') as json_file:
    dataset = json.load(json_file)

### Setup Model

In [22]:
llm = Llama(model_path="llama-2-13b.Q4_0.gguf", seed=SEED, n_gpu_layers=1, n_ctx=CONTEXT_SIZE, verbose=False)

llama_model_loader: loaded meta data with 19 key-value pairs and 363 tensors from llama-2-13b.Q4_0.gguf (version GGUF V2 (latest))
llama_model_loader: - tensor    0:                token_embd.weight q4_0     [  5120, 32000,     1,     1 ]
llama_model_loader: - tensor    1:           blk.0.attn_norm.weight f32      [  5120,     1,     1,     1 ]
llama_model_loader: - tensor    2:            blk.0.ffn_down.weight q4_0     [ 13824,  5120,     1,     1 ]
llama_model_loader: - tensor    3:            blk.0.ffn_gate.weight q4_0     [  5120, 13824,     1,     1 ]
llama_model_loader: - tensor    4:              blk.0.ffn_up.weight q4_0     [  5120, 13824,     1,     1 ]
llama_model_loader: - tensor    5:            blk.0.ffn_norm.weight f32      [  5120,     1,     1,     1 ]
llama_model_loader: - tensor    6:              blk.0.attn_k.weight q4_0     [  5120,  5120,     1,     1 ]
llama_model_loader: - tensor    7:         blk.0.attn_output.weight q4_0     [  5120,  5120,     1,     1 ]
llama

In [27]:
prompt = "Die FDP ist eine Partei, die"

In [28]:
output = llm(prompt, max_tokens=MAX_TOKENS, stop=["Q:", "\n"], echo=True)
print(output)

{'id': 'cmpl-f1238d19-4f7a-4aa0-8229-7811a49e3540', 'object': 'text_completion', 'created': 1695027955, 'model': 'llama-2-13b.Q4_0.gguf', 'choices': [{'text': 'Die FDP ist eine Partei, die auf den Verhaltenskodex der FDP Deutschland verpflichtet wurde. In allen Regionen werden wir auf Grundlage von Vorschriften des jeweiligen Landes zertifiziert und regelmäßig aufrecht gehalten.', 'index': 0, 'logprobs': None, 'finish_reason': 'stop'}], 'usage': {'prompt_tokens': 9, 'completion_tokens': 50, 'total_tokens': 59}}
